In [1]:
from os.path import dirname, join
import re

import numpy as np
import pandas as pd

from elvis import datasets

from elvis.io.boem_from_file import (boem_lease_by_owner, 
                                     boem_leases, 
                                     boem_lease_owner,
                                     contours_to_geojson,
                                     freeze_bids,
                                     get_blocks_by_owner,
                                     get_current_leases,
                                     winning_bid_by_block)

base_directory = dirname(datasets.__file__)

# supplied data
freeze_data = join(base_directory, 'Freeze_Data\ 12_4_2019')

#
base_directory = dirname(datasets.__file__)

### Questions about the bid and lease data?

1. Does the "Lease Status Code" get updated periodically? 

Relevance - If the lease status is "PROD" for production, then 
there is likely a preference for an agent to accumulate other 
lease blocks in the vacinity. 

2. Can we get multiple snapshots of the lease owners, to track the change 
   of ownership over time. Also reveals the activity of flipping leases.
   Similarly for pipeline maps and other infrastructure.
   
Relevance - We can see how the lease owners change. This might characterize 
smaller players aquire existing leases. The behaviour of larger players in 
divesting; and perhaps moving the center of mass of their operations to 
deeper waters? Having an evolving view of the infrastructure would also give 
us a meaningful metric of "distance to pipeline". 

### Todo

include field, and well information.

### Features

1. Proximity to infrastructure
2. [Proximity to producing wells].
3. depth.
4. Proximity to other leases (on aggregate, by company)
5. Inventory building  #blocks - expiry; if they are hording. 
   Exxon/BHP/Total reduced; function of oil price as well.
   
*2010/2011 pause new leases, Macondo?

6. Need timeseries for oil price yearly, ideally get forecast?
   Current year, 5yrs from now - make a basic model.

7. (binary) Operator/Major working (bidding) in the neighbourhood? - Money and knowledge. 
   neighbourhood - historically active, concentration of bidding (std. dev 1-block).

(100-features)

### QC

Histogram description of the time series data.
Geospatial data.

### Next steps

Price = \sum factors
[Just to the expectation]
*Range p10/p90 for price. 

Probility bid - logistic regression 

Color code - intensity of competition (plausibility, how bad people need the land).
(GOM-3 inverse of company probability number.)

### GOM-Cubed/Meeting 3 Feb.

Look for features the GOM-cubed used. Look at ESA bid data set from slack channel.



   

In [4]:
bid_data.loc["WR271"]

,OID_,SALEDATE,SALENUM,BIDORDER,BID,Company Name,ESASORT,PCTSHARE,MROV,ADV,RAM,Consortia
Lease Number,,,,,,,,,,,,
G23042,NaN,2001-03-28,178-1,1,261800.0,kerrmcgeeoilgascorporation,KERR-MCGEE,100.00,216000.0,216000.0,0.0,False
G35080,NaN,2013-03-20,227,1,81787999.0,statoilgulfofmexicollc,STATOIL,66.67,34000000.0,25000000.0,0.0,True
G35080,NaN,2013-03-20,227,2,17545960.0,shelloffshoreinc,SHELL,100.00,34000000.0,25000000.0,0.0,False


In [3]:
bid_data, winning_bids, consortia_bids = freeze_bids(freeze_data)
bid_by_block = winning_bid_by_block(base_directory, freeze_data)

In [5]:
bid_by_block

,,BLOCKS_,BLOCKS_ID,MMS_REGION,MMS_PLAN_A,PROT_NUMBE,PROT_APRV_,BLOCK_NUMB,BLK_FED_AP,BLOCK_LAB,AREA_CODE,...,SALENUM,BIDORDER,BID,Original Bidder,ESASORT,PCTSHARE,MROV,ADV,RAM,Bid by Consortia
AREABLK,Lease Number,,,,,,,,,,,,,,,,,,,,,
WC17,G01351,1,1,G,None,LA1,01-JUL-2011,17,01-JUL-2011,17,WC,...,012,1,2108385.0,californiaoilcompany,CHEVRON,100.0,0.0,0.0,0.0,False
WC18,G01435,2,2,G,None,LA1,01-JUL-2011,18,01-JUL-2011,18,WC,...,014,1,3257550.0,texacoinc,TEXACO,100.0,0.0,0.0,0.0,False
WC21,G23730,5,5,G,None,LA1,01-JUL-2011,21,01-JUL-2011,21,WC,...,182,1,311455.0,dukeenergyhydrocarbonsllc,DUKE ENERGY,63.0,62325.0,62325.0,0.0,True
WC33,G15050,16,16,G,None,LA1,01-JUL-2011,33,01-JUL-2011,33,WC,...,152,1,133375.0,mobiloilexplorationproduc,MOBIL,100.0,0.0,0.0,0.0,False
WC35,G01860,18,18,G,None,LA1,01-JUL-2011,35,01-JUL-2011,35,WC,...,019,1,2002000.0,atlanticrichfieldcompany,ARCO,50.0,0.0,0.0,0.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
HI179,G03236,28452,28452,G,None,TX7,01-NOV-2000,179,01-NOV-2000,179,HI,...,38A,1,8585000.0,shelloilcompany,SHELL,100.0,0.0,0.0,0.0,False
HI193,G03237,28453,28453,G,None,TX7,01-NOV-2000,193,01-NOV-2000,193,HI,...,38A,1,4798000.0,exxoncorporation,EXXON,100.0,0.0,0.0,0.0,False
HI206,G20660,28466,28466,G,None,TX7,01-NOV-2000,206,01-NOV-2000,206,HI,...,171,1,1171552.0,senecaresourcescorporation,SENECA,100.0,144000.0,144000.0,0.0,False


In [6]:
changed_hands = bid_by_block['Original Bidder'] != bid_by_block['Company Name']
bid_by_block = bid_by_block[changed_hands]
# since 2000
bid_by_block = bid_by_block[bid_by_block['Asgn Eff Date'] > pd.datetime(2010,1,1)]

In [8]:
changed_hands['WR272']

Lease Number
G35081    True
dtype: bool

In [ ]:
WR272

In [37]:
bid_by_block.xs(key='Lease Number', level=1)

KeyError: 'Lease Number'

In [33]:
help(bid_by_block.xs)

Help on method xs in module pandas.core.generic:

xs(key, axis=0, level=None, drop_level=True) method of geopandas.geodataframe.GeoDataFrame instance
    Return cross-section from the Series/DataFrame.
    
    This method takes a `key` argument to select data at a particular
    level of a MultiIndex.
    
    Parameters
    ----------
    key : label or tuple of label
        Label contained in the index, or partially in a MultiIndex.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Axis to retrieve cross-section on.
    level : object, defaults to first n levels (n=1 or len(key))
        In case of a key partially contained in a MultiIndex, indicate
        which levels are used. Levels can be referred by label or position.
    drop_level : bool, default True
        If False, returns object with same levels as self.
    
    Returns
    -------
    Series or DataFrame
        Cross-section from the original Series or DataFrame
        corresponding to the selected index

In [32]:
bid_by_block.xs("AREABLK")

KeyError: 'AREABLK'

In [17]:
bid_by_block['Company Name']

Lease Number  AREABLK
G01351        WC17             coxoiloffshorellc
G01435        WC18             coxoiloffshorellc
G23730        WC21       sanareenergypartnersllc
G15050        WC33            fieldwoodenergyllc
G01860        WC35            fieldwoodenergyllc
                                  ...           
G27509        HI176      walteroilgascorporation
G03236        HI179           fieldwoodenergyllc
G03237        HI193                arenaenergylp
G20660        HI206           fieldwoodenergyllc
G01848        HI129           fieldwoodenergyllc
Name: Company Name, Length: 1085, dtype: object

Display blocks that have changed hands in the last 10 years. 

*FIXME - This is not inclusive, only that the block has a different bidder vs owner

In [12]:
from elvis.visualization.mapping import (bathymetry_underlay,
                                         create_map_from_geojson,
                                         colors as company_colors,
                                         geojson_underlay)

bathymetry = contours_to_geojson(join(base_directory, 
                                      "Gulf_Bathymetric_Contours"),
                                      "contours_noaa_500m")
contours = geojson_underlay(bathymetry, 
                            name="bathymetry",
                            color='black',
                            weight=0.25)

create_map_from_geojson(bid_by_block.__geo_interface__,
                        color=company_colors["equinor"],
                        underlays=contours)

ShapefileException: Unable to open /Users/blasscock/Documents/Code/equinor/elvis/datasets/Gulf_Bathymetric_Contours/contours_noaa_500m.dbf or /Users/blasscock/Documents/Code/equinor/elvis/datasets/Gulf_Bathymetric_Contours/contours_noaa_500m.shp.

### Top-10's
Plot up the top-10 highest bid lease blocks by year (or like 5-years). 

In [ ]:
periods = pd.date_range(start=pd.datetime(1950,1,1), end=pd.datetime(2020,1,1), periods=8)

colors = ["red", "green", "yellow", "blue", "pink", "orange", "cyan"]


indx = np.argsort(bid_by_block["BID"])[::-1]
all_time_top10 = bid_by_block.iloc[indx[:10]]

underlays = []
for i in range(1,len(periods)):
    tmin, tmax = periods[i-1], periods[i]
    
    indx = np.logical_and(bid_by_block["SALEDATE"] > tmin, bid_by_block["SALEDATE"] < tmax)
    _bid_by_block = bid_by_block[indx]
    
    indx = np.argsort(_bid_by_block["BID"])[::-1]
    _top_tens = _bid_by_block.iloc[indx[:10]]
    
    _underlays = geojson_underlay(_top_tens.__geo_interface__, 
                                  name='period {}'.format(i),
                                  color=colors[i-1],
                                  fillOpacity=1.0,
                                  weight=0.25)
    underlays.append(_underlays)

    


In [13]:
create_map_from_geojson(bathymetry,
                        color="black",
                        underlays=underlays,
                        opacity=1.0)

Map(center=[26.9792212296875, -91.87030927187499], controls=(ZoomControl(options=['position', 'zoom_in_text', …